In [2]:
import tensorflow as tf

import numpy as np 
import time
import os
from tensorflow.python.keras.datasets import cifar10  
from keras.utils import np_utils
from tensorflow.python import debug as tf_debug

import warnings
warnings.filterwarnings('ignore')

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

num_classes = np.unique(y_train).shape[0]

y_train = y_train.reshape(y_train.shape[0])  
y_test = y_test.reshape(y_test.shape[0])

x_train = X_train.astype('float32') / 255
x_test = X_test.astype('float32') / 255

x_train = np.reshape(x_train, [-1, 3072])
x_test  = np.reshape(x_test, [-1, 3072])

y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [4]:
def weight_variable(shape): # создание переменной весов слоя
    # тензор определенной формы, заполненный случайными 
    # усеченными нормальными значениями
    initial = tf.truncated_normal(shape, stddev=0.1) 
    return tf.Variable(initial)

def bias_variable(shape): # Отступ для полносвязного или свёрточного слоя
    initial = tf.constant(0.1, shape=shape) 
    return tf.Variable(initial)

def conv2d(x_, W): # операция свертки
    return tf.nn.conv2d(x_, W, strides=[1, 1, 1, 1], padding='SAME')

# операция объединения (уменьшает вдвое размер сверточного слоя) (пулинга)
def max_pool_2x2(x_):  
    return tf.nn.max_pool(x_, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def conv_layer(input, shape):  # сверточный слой
    W = weight_variable(shape) # переменная для весов свертки
    # board
    variable_summaries(W)
    b = bias_variable([shape[3]]) # свободные члены = количество фильтров
    # board
    variable_summaries(b)
    return tf.nn.relu(conv2d(input, W) + b)

def full_layer(input, size):  # Функция создания полносвязного слоя
    in_size = int(input.get_shape()[1]) 
    W = weight_variable([in_size, size]) 
    # board
    variable_summaries(W) 
    b = bias_variable([size]) 
    # board
    variable_summaries(b) 
    return tf.matmul(input, W) + b

In [5]:
# Функция из документации Tensorflow для журналирования сводок 
# board
def variable_summaries(var):
    with tf.name_scope('summaries'): 
        mean = tf.reduce_mean(var) 
        tf.summary.scalar('mean', mean) 
        with tf.name_scope('stddev'): 
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean))) 
        tf.summary.scalar('stddev', stddev) 
        tf.summary.scalar('max', tf.reduce_max(var)) 
        tf.summary.scalar('min', tf.reduce_min(var)) 
        tf.summary.histogram('histogram', var)

LOG_DIR = "dir_lab4/logs/summaries" 
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 3072])   # создаем переменную для x
y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)   # переменная для dropout

x_image = tf.reshape(x, [-1, 32, 32, 3])   # преобразование в размеры картинки

In [6]:
with tf.name_scope('conv_1'):
    conv1 = conv_layer(x_image, shape=[3, 3, 3, 32]) # создание сверточного слоя      
    conv1_pool = max_pool_2x2(conv1) # слой пулинга -> Уменьшаем выходные данные в 2 раза
with tf.name_scope('conv_2'): 
    conv2 = conv_layer(conv1_pool, shape=[3, 3, 32, 64]) 
    conv2_pool = max_pool_2x2(conv2) # слой пулинга -> Уменьшаем выходные данные в 2 раза
with tf.name_scope('conv_3'): 
    conv3 = conv_layer(conv2_pool, shape=[3, 3, 64, 128]) 
    conv3_pool = max_pool_2x2(conv3) # слой пулинга -> Уменьшаем выходные данные в 2 раза
    conv3_flat = tf.reshape(conv3_pool, [-1, 4*4*128]) 
with tf.name_scope('full_1'):
    # computes rectified linear unit 
    full_1 = tf.nn.relu(full_layer(conv3_flat, 512)) 
with tf.name_scope('dropout'):
    # to prevent overfitting 
    full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob) 
with tf.name_scope('activations'): 
    y_conv = full_layer(full1_drop, 10) 
    variable_summaries(y_conv)
with tf.name_scope('cross'): 
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv, labels=y_))# cost
    tf.summary.scalar('cross_entropy', cross_entropy) 
#SGD 
train_step = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1)) 
with tf.name_scope('accuracy'): 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32)) # средняя точность 
    tf.summary.scalar('accuracy', accuracy) # собираем вывод

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# Мерджим все итоги
merged = tf.summary.merge_all()

In [8]:
# saver
DIR = 'dir_lab4/model'
train_var = [x, y_, accuracy, keep_prob]
tf.add_to_collection('train_var', train_var[0]) 
tf.add_to_collection('train_var', train_var[1]) 
tf.add_to_collection('train_var', train_var[2]) 
tf.add_to_collection('train_var', train_var[3]) 
saver = tf.train.Saver(max_to_keep=7, keep_checkpoint_every_n_hours=0.5)
saver.export_meta_graph(os.path.join(DIR, "model_ckpt.meta"), 
                        collection_list=['train_var'])

meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "ApplyAdam"
      input_arg {
        name: "var"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "m"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "v"
        type_attr: "T

In [9]:
with tf.Session() as sess:
    
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train', graph=tf.get_default_graph()) 
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test', graph=tf.get_default_graph())
    
    sess.run(tf.global_variables_initializer()) 
    start_time = time.time()
    num_epochs = 4
    for epochs in range(num_epochs):
        for i in range(2800): 
            batch = [x_train[25*i:25*(i+1)] , y_train[25*i:25*(i+1)]]# each batch of 25 samples 
            if i % 200 == 0: 
                summary,train_accuracy = sess.run([merged,accuracy], 
                                                  feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}) 
                print("Время {}, шаг {} точность обучения {}"
                      .format(time.time() - start_time, i, train_accuracy))
                # Организовываем вывод
                # board
                train_writer.add_summary(summary, i+2800*epochs) 
            sess.run(train_step, feed_dict={x: batch[0], y_: batch[1],keep_prob: 0.5})
        print('-----------------------------------------------Эпоха№', epochs)
        # Сохраняем
        saver.save(sess, os.path.join(os.getcwd(), 'lab4'))

    X = x_test.reshape(10, 1000, 3072) 
    Y = y_test.reshape(10, 1000, 10) 

    final_test_accuracy = []
    for i in range(10):
        summary, test_accur = sess.run([merged, accuracy], feed_dict={x:X[i], y_:Y[i], keep_prob:1.0})
        final_test_accuracy.append(test_accur)
        # board
        test_writer.add_summary(summary, i)                    
    print("Проверяемая точность (тестируемая): {}".format(np.mean(final_test_accuracy)))

Время 0.14586949348449707, шаг 0 точность обучения 0.0
Время 9.494699716567993, шаг 200 точность обучения 0.20000000298023224
Время 19.27668786048889, шаг 400 точность обучения 0.4000000059604645
Время 29.834085941314697, шаг 600 точность обучения 0.47999998927116394
Время 41.571696043014526, шаг 800 точность обучения 0.36000001430511475
Время 50.656413555145264, шаг 1000 точность обучения 0.4000000059604645
Время 60.01618957519531, шаг 1200 точность обучения 0.5600000023841858
Время 69.67698049545288, шаг 1400 точность обучения 0.47999998927116394
Время 79.42128467559814, шаг 1600 точность обучения 0.5199999809265137
Время 89.34710359573364, шаг 1800 точность обучения 0.7599999904632568
Время 97.93441486358643, шаг 2000 точность обучения nan
Время 99.27787661552429, шаг 2200 точность обучения nan
Время 100.60875344276428, шаг 2400 точность обучения nan
Время 101.80929231643677, шаг 2600 точность обучения nan
-----------------------------------------------Эпоха№ 0
Время 103.80254960060

При возникновении ошибки 
>NotFoundError: Key W_1 not found in checkpoint [[Node: save/RestoreV2_2 = RestoreV2[
dtypes=[DT_FLOAT], _device="/job:localhost/replica:0 /task:0/cpu:0"](_recv_save/Const_1_0,
save/RestoreV2_2 /tensor_names, save/RestoreV2_2/shape_and_slices)]]

нужно сбросить граф по умолчанию:  
`tf.reset_default_graph()`